In [1]:
import numpy as np

In [2]:
import random

In [3]:
default_projection =np.zeros(1,) # Default projection vector, just to make things work 

In [4]:
def get_ground_state(num_qubits):
    
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    state_vector = np.zeros(2**num_qubits)
    state_vector[0] = 1
    
    return state_vector

def get_operator(total_qubits,projection, gate_unitary, target_qubit):
    
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    
    Matrix = 0
    
    if projection.shape == (1,):
        Matrix = gate_unitary
    else :
        Matrix = projection
        
    I = np.identity(2)
    K = I
    flag = False # this is used so that gate_unitary or projection will be at the correct position
                 # during the tensor product
    
    for i in range(total_qubits-2):
            if target_qubit == 0:
                K = np.kron(Matrix,K)
                flag = True
                continue
            elif i==target_qubit-1:
                K = np.kron(K,Matrix)
                flag = True
                continue
            K = np.kron(I,K)
             
    if flag:
        Operator = np.kron(K,I)
    else:    
        Operator = np.kron(K,Matrix)
    
    flag = False
    
    return Operator

def cnot_operator(total_qubits, gate_unitary, target_qubits):
    
    # X gate (CNOT is controlled-X):
    gate_unitary = np.array([
                        [0, 1],
                        [1, 0]
                    ])
    
    # Projection operator |0><0|
    P0x0 = np.array([
            [1, 0],
            [0, 0]
         ])

    # Projection operator |1><1|
    P1x1 = np.array([
            [0, 0],
            [0, 1]
         ])
    
    
    Operator_1 = 0
    Operator_2a = 0
    Operator_2b = 0
    Controlled_qubit = target_qubits[0] 
    target_qubit = target_qubits[1]
    
    # if the controlled qubit is |0>
    Operator_1 = get_operator(total_qubits, P0x0,gate_unitary,Controlled_qubit)
    
    # if the controlled qubit is |1>
    Operator_2a = get_operator(total_qubits,P1x1, gate_unitary,Controlled_qubit)
    Operator_2b = get_operator(total_qubits,default_projection, gate_unitary,target_qubit)
        
    return Operator_1+(np.dot(Operator_2a,Operator_2b))

def run_program(initial_state, program):
    
    
    Gates = [] # List to store all the gates/unitaries seprately
    Operators = [] # To store all operators
    qubits = [] # To store the target qubits 
    final_state = initial_state
    total_qubits = int(np.log2(len(initial_state))) # Total number of qubits
    
    # read program, and for each gate:
    
    for i in range(len(program)):
        Gates.append(program[i]["unitary"])
        qubits.append(program[i]["target"])
        
    #   - calculate matrix operator
     
    for i in range(len(Gates)):  
        if len(qubits[i])==2 :
            Operators.append(cnot_operator(total_qubits,Gates[i],qubits[i]))
        else :
            Operators.append(get_operator(total_qubits,default_projection,Gates[i],qubits[i][0]))
        
    #   - multiply state with operator
    for i  in range(len(Operators)):
         final_state = np.dot(final_state,Operators[i])
            
    # return final state
    return final_state

def measure_all(state_vector):
    
    # choose element from state_vector using weighted random and return it's index
    
    num_qubits = int(np.log2(len(state_vector))) # Total number of qubits
    
    # this will create a list of length equal to the length of state_vector 
    # and elements will be index number in binary form. eg: num_qubits = 2 => index_list = ['00', '01', '10', '11']
    
    index_list = list((bin(i).replace("0b", "")).zfill(num_qubits) for i in range(len(state_vector))) 
    
    key = random.choices(index_list,weights=state_vector,k=1)[0]
    
    return key

def get_counts(state_vector, num_shots):
    
    # simply execute measure_all in a loop num_shots times and
    
    Counts = {} # To store the element_index & number_of_ocurrences 
    
    
    for shot in range(num_shots):
        key = measure_all(state_vector)
        if key in Counts :
            Counts[key] = 1+Counts.get(key)
        else:
            Counts[key] = 1 
   
    return Counts

In [5]:
X = np.array([
[0, 1],
[1, 0]
])

In [6]:
my_circuit = [
  { "unitary": [[0.70710678, 0.70710678], [0.70710678, -0.70710678]], "target": [0] }, 
  { "unitary": [ [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0] ], "target": [0, 1] }
]

In [7]:
initial_state = get_ground_state(3)

In [8]:
initial_state

array([1., 0., 0., 0., 0., 0., 0., 0.])

In [9]:
final_state=run_program(initial_state,my_circuit)

In [10]:
final_state

array([0.70710678, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.70710678, 0.        ])

In [11]:
get_counts(final_state,1000)

{'110': 499, '000': 501}